In [3]:
import pandas as pd
from pyvi import ViTokenizer
import re
import nltk
import seaborn as sns
from matplotlib import pyplot

# 1. Import data

In [24]:
filename = "train_nor_811.xlsx"
data = pd.read_excel(filename, engine = "openpyxl")

In [25]:
data.head()

,Unnamed: 0,Emotion,Sentence
0,188,Other,cho mình xin bài nhạc tên là gì với ạ
1,166,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,1345,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,316,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,1225,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...


In [26]:
data.drop(columns = {"Unnamed: 0"}, axis = 1, inplace = True)
data.head()

,Emotion,Sentence
0,Other,cho mình xin bài nhạc tên là gì với ạ
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :))
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...


In [27]:
from sklearn.preprocessing import LabelEncoder
data["emotion_encode"] = data["Emotion"]
encoder = LabelEncoder()
data.emotion_encode = encoder.fit_transform(data.Emotion)
data.head()

,Emotion,Sentence,emotion_encode
0,Other,cho mình xin bài nhạc tên là gì với ạ,4
1,Disgust,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,1
2,Disgust,lo học đi . yêu đương lol gì hay lại thích học...,1
3,Enjoyment,uớc gì sau này về già vẫn có thể như cụ này :)),2
4,Enjoyment,mỗi lần có video của con là cứ coi đi coi lại ...,2


# 2. Data visualization

# 3. Data preprocessing

In [28]:
clean_sentences = []
for i in range(len(data)):
    clean_sentences.append(ViTokenizer.tokenize(data.Sentence[i]))

In [29]:
import json
f = open("sensitive_words.json", encoding="utf-8")
sensitive_words = json.load(f)
f.close()

In [30]:
def remove_duplicate(word):
    prev_char = ""
    clean_word = ""
    for character in word:
        if(character != prev_char):
            clean_word += character
            prev_char = character
    return clean_word

In [31]:
def deEmojify(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def normalize_sentences(sentences):
    punc_lst = {'.', ',', '...', '-', '“', '”', ':', '(', ')', '"', '!', '&', ';', '?', '*', ']', '>', '…', '’',"``","''", "=", "%", "^", "@", "<", ">"}
    confusing_words = {"per", "dume"}
    acronym_word = {
        "ko" : "không",
        "k" : "không",
        "z" : "vậy",
        "dzậy" : "vậy",
        "dậy": "vậy",
        "t" : "tao",
        "m" : "mày",
        "sgk" : "sách_giáo_khoa",
        "zi" : "vậy",
        "dth" : "dễ_thương"
        
    }
    clean_sentences = []
    
    # remove punctuation and lowercase
    for sent in sentences:
        
        # remove emojis
        sent = deEmojify(sent)
        
        sent = nltk.word_tokenize(sent)
        temp = []
        for word in sent:
            word = word.lower()
            word = remove_duplicate(word)
            if (word in punc_lst or word in confusing_words):
                continue
            elif(word in acronym_word):
                temp.append(acronym_word[word])
            elif(word in sensitive_words):
                temp.append("<SEN>")
            elif(word.isdigit()):
                temp.append("<NUM>")
            else:
                temp.append(word)
        # remove whitespace
        sent = ' '.join(temp)
        
        clean_sentences.append(sent)
        
    return clean_sentences

In [32]:
clean_sentences = normalize_sentences(clean_sentences)
encode_tags = data.emotion_encode

In [33]:
# remove empty sentences
for idx, sent in enumerate(clean_sentences):
    if sent.strip() == "":
        del clean_sentences[idx]
        del encode_tags[idx]

In [34]:
clean_sentences

['cho mình xin bài nhạc tên là gì với ạ',
 'cho đáng_đời con quỷ về nhà lôi con nhà <SEN> ra mà đánh',
 'lo học đi yêu_đương <SEN> gì hay lại thích học_sinh_học',
 'uớc gì sau_này về già vẫn có_thể như cụ này',
 'mỗi lần có video của con là cứ coi đi coi lại hoài cưng con quá',
 'thằng kia sao <SEN> bắt vợ với bồ tao dọn thế kia nhà <SEN> ở đâu tao đến thịt <SEN> chết',
 'một lí_do trog muôn_vàn lí_do',
 'thật hay đùa ác_vậy không_thể tin được',
 'không phải con mình mà xem còn thấy đau như vậy huống gì người trong cuộc thật là phẫn_nộ mà cơ_quan_chức_năng làm_việc quá chậm_trễ đến giờ mà vẫn chưa tìm ra người chịu trách_nhiệm',
 'nghe đi rồi khóc <NUM> trận cho thoải_mái đừng cố gồng mình lên nữa',
 'công_nhận sáng_tạo thật đấy',
 'đòn tấn_công cực gắt và cục sút của anh',
 'trời nắng_nóng thế này mình muốn bán_nước không biết thu_nhập có cao không ạ <NUM>',
 'minh biết nữa ne',
 'mấy thằng củ <SEN> việt nam nhảm_nhí',
 'tui thi ́ ch va ̉ i lă ́ <SEN> ma ̀ ăn nhỉ ̀ u no ́ người lă ́ <

# 4. Model architecture

In [35]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras import layers, activations , models , preprocessing , utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [36]:
MAX_LEN = 150

tokenizer = Tokenizer(lower = True, split = ' ')
tokenizer.fit_on_texts(clean_sentences)

X = tokenizer.texts_to_sequences(clean_sentences)

X = pad_sequences(X, MAX_LEN, padding='post', truncating='post')

print(X.shape)

(5547, 150)


In [37]:
vocab_size = len(tokenizer.word_index) + 1

In [38]:
from  sklearn.model_selection import train_test_split
Y = tf.keras.utils.to_categorical(encode_tags)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 36)

## 4.1 CNN Model

In [39]:
embedding_dim = 128

inputs = layers.Input(shape=( MAX_LEN , ))
embedding = layers.Embedding(vocab_size, embedding_dim, input_length=MAX_LEN)(inputs)

cnn1 = layers.Conv1D(filters=100, kernel_size=1, activation='relu')(embedding)
cnn1 = layers.MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

cnn2 = layers.Conv1D(filters=100, kernel_size=2, activation='relu')(embedding)
cnn2 = layers.MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

outputs = layers.Concatenate()([cnn1,cnn2])

outputs = layers.Dense(28, activation='tanh')(outputs)
outputs = layers.Dense(14, activation='tanh')(outputs)
outputs = layers.Dense(7, activation='softmax')(outputs)
model=models.Model(inputs,outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 150, 128)     501760      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 150, 100)     12900       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 149, 100)     25700       embedding[0][0]                  
_______________________________________________________________________________________

In [40]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1)
history = model.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_test, y_test),
                    batch_size=64)

Epoch 1/10
65/65 [==============================] - 7s 109ms/step - loss: 0.3776 - accuracy: 0.2911 - val_loss: 0.3728 - val_accuracy: 0.3151
Epoch 2/10
65/65 [==============================] - 6s 95ms/step - loss: 0.3233 - accuracy: 0.4512 - val_loss: 0.3303 - val_accuracy: 0.4376
Epoch 3/10
65/65 [==============================] - 7s 105ms/step - loss: 0.2444 - accuracy: 0.6279 - val_loss: 0.3413 - val_accuracy: 0.4420
Epoch 00003: early stopping


## 4.2 LSTM Model

In [120]:
from keras.models import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense

model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LEN))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(7, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [121]:
history2 = model2.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_test, y_test),
                    batch_size=512)

Epoch 1/10
16/16 [==============================] - 78s 5s/step - loss: 1.9476 - accuracy: 0.1412 - val_loss: 1.9463 - val_accuracy: 0.1430
Epoch 2/10
16/16 [==============================] - 91s 6s/step - loss: 1.9473 - accuracy: 0.1384 - val_loss: 1.9463 - val_accuracy: 0.1434
Epoch 3/10
16/16 [==============================] - 109s 7s/step - loss: 1.9468 - accuracy: 0.1424 - val_loss: 1.9460 - val_accuracy: 0.1434
Epoch 4/10
16/16 [==============================] - 124s 8s/step - loss: 1.9463 - accuracy: 0.1366 - val_loss: 1.9461 - val_accuracy: 0.1390
Epoch 00004: early stopping


## 4. BiLSTM model

In [125]:
bi_model = Sequential()
bi_model.add(Embedding(vocab_size, embedding_dim, input_length=MAX_LEN))
bi_model.add(SpatialDropout1D(0.2))
bi_model.add(layers.Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2)))
bi_model.add(Dense(7, activation='softmax'))
bi_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [126]:
bi_history = bi_model.fit(X_train, y_train,
                    epochs=10,
                    callbacks = [es],
                    validation_data=(X_test, y_test),
                    batch_size=64)

Epoch 1/10
128/128 [==============================] - 148s 1s/step - loss: 1.9131 - accuracy: 0.1856 - val_loss: 1.8468 - val_accuracy: 0.2743
Epoch 2/10
128/128 [==============================] - 144s 1s/step - loss: 1.7431 - accuracy: 0.3233 - val_loss: 1.7461 - val_accuracy: 0.3183
Epoch 3/10
128/128 [==============================] - 162s 1s/step - loss: 1.5398 - accuracy: 0.4172 - val_loss: 1.7427 - val_accuracy: 0.3319
Epoch 4/10
128/128 [==============================] - 148s 1s/step - loss: 1.3770 - accuracy: 0.4888 - val_loss: 1.8018 - val_accuracy: 0.3330
Epoch 00004: early stopping


In [49]:
bi_model.save("bi_model.h5")

## 4.3 Decision Tree Model + TF IDF vectorize

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [42]:
# TF IDF vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf = True,max_features=3000)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(clean_sentences)
print(tfidf_vectorizer_vectors.shape)

(5547, 3000)


In [43]:
tree_model = DecisionTreeClassifier()
Y_DT = encode_tags

kf = KFold(n_splits=10)
for train_index, test_index in kf.split(tfidf_vectorizer_vectors):
    X_train_DT, X_test_DT = tfidf_vectorizer_vectors[train_index], tfidf_vectorizer_vectors[test_index]
    y_train_DT, y_test_DT = Y_DT[train_index], Y_DT[test_index]
    
    tree_model.fit(X_train_DT, y_train_DT)

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([4141], dtype='int64'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [46]:
y_pred_DT = tree_model.predict(X_test_DT)

In [47]:
print(classification_report(y_test_DT, y_pred_DT))

              precision    recall  f1-score   support

           0       0.19      0.23      0.21        39
           1       0.40      0.42      0.41       101
           2       0.47      0.43      0.45       173
           3       0.39      0.27      0.32        33
           4       0.22      0.23      0.22       101
           5       0.32      0.39      0.35        82
           6       0.25      0.16      0.20        25

    accuracy                           0.35       554
   macro avg       0.32      0.30      0.31       554
weighted avg       0.36      0.35      0.35       554



## 4.4 Linear SVC + TF IDF vectorize

In [47]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [48]:
from  sklearn.model_selection import train_test_split
Y_DT = encode_tags
X_train_DT, X_test_DT, y_train_DT, y_test_DT = train_test_split(tfidf_vectorizer_vectors, Y_DT, test_size = 0.25, random_state = 36)

In [49]:
linear_svc = LinearSVC()
clf = linear_svc.fit(X_train_DT,y_train_DT)

svc_model = CalibratedClassifierCV(base_estimator=linear_svc, cv="prefit")

svc_model.fit(X_train_DT,y_train_DT)
y_pred_DT = svc_model.predict(X_test_DT)

In [50]:
print(classification_report(y_test_DT, y_pred_DT))

              precision    recall  f1-score   support

           0       0.39      0.33      0.36        96
           1       0.49      0.49      0.49       267
           2       0.60      0.60      0.60       406
           3       0.68      0.48      0.57        93
           4       0.30      0.35      0.32       234
           5       0.48      0.53      0.50       219
           6       0.43      0.28      0.34        72

    accuracy                           0.48      1387
   macro avg       0.48      0.44      0.45      1387
weighted avg       0.49      0.48      0.48      1387

